In [40]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

chain.predict(
    question="My name is Ben",
)




> Entering new LLMChain chain...
Prompt after formatting:
My name is Ben

> Finished chain.


'Nice to meet you, Ben! How can I assist you today?'

In [41]:
chain.predict(
    question="I live in Seoul",
)




> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [42]:
chain.predict(
    question="What is my name?",
)




> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name."

In [43]:
memory.load_memory_variables({})

{'history': "System: Ben introduces himself and mentions that he lives in Seoul. The AI responds by acknowledging Seoul as a vibrant city with a rich history and culture, asking Ben what he enjoys most about living there.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}